# Nutzung existierender Embeddings

Das Training eigener Embeddings hast du bereits kennengelernt. Um Embeddings trainieren zu können, benötigst du eine relativ große Datenmenge, die dir möglicherweise nicht immer zur Verfügung steht.

Für unterschiedliche Anwendungsfälle und verschiedene Sprachen kannst du auch bereits existierende Embeddings herunterladen und mit diesen arbeiten. Damit kannst du z.B. Landessprachen erkennen oder Wortähnlichkeiten ermitteln, ohne die Embeddings zuvor aus einer großen Textdatenmenge lernen zu müssen.

## Erkennung der Landesprache

Mit der Landesspracherkennung hast du bereits gearbeitet, daher ist diese hier nur der Vollständigkeit halber erwähnt. Als  Technologie wird hier nämlich auch ein Embedding-Verfahren benutzt, allerdings in Zusammenhang mit einer Klassifikation (die du nun auch bereits kennst).

Nähere Infos findest du unter https://fasttext.cc/docs/en/language-identification.html

## Deutschsprachige Embeddings

### Facebook - Common Crawl

Deutschsprachige Embeddings kannst du z.B. von https://fasttext.cc/docs/en/crawl-vectors.html herunterladen, da ist dann die Datei https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz besonders interessant. Wenn du Colab benutzt, musst du nichts tun. Sonst lade dir bitte die Datei herunter und entpacke sie.

Diese Embeddings kannst du nicht mit `gensim` einladen, sondern du musst dazu das `fasttext` Paket nutzen:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f cc.de.300.bin || wget  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.bin.gz && gunzip cc.de.300.bin.gz")

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
cc_ft = fasttext.load_model("cc.de.300.bin")

Mit dem großen Modell kannst du Wortähnlichkeiten bestimmen: 

In [ ]:
cc_ft.get_nearest_neighbors("apfel", k=20)

In [ ]:
cc_ft.get_nearest_neighbors("Apfel", k=20)

Oder auch die immer wieder zitierten Analogieschlüsse ausprobieren:

In [ ]:
cc_ft.get_analogies("Frau", "Mann", "König")

In [ ]:
cc_ft.get_analogies("Frau", "Mann", "Onkel")

In [ ]:
# Speicher freigeben für Colab
del cc_ft

### deepset-Embeddings

Es gibt noch deutlich mehr "fertige" Embeddings, z.B. von https://deepset.ai/german-word-embeddings:
* https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt (glove), bitte umbennen in `emb-glove-de-wiki.txt`
* https://int-emb-word2vec-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt (word2vec), bitte umbennen in `emb-word2vec-de-wiki.txt`
* https://s3.eu-central-1.amazonaws.com/int-emb-fasttext-de-wiki/20180917/model.bin (fastText), bitte umbennen in `emb-fasttext-de-wiki.bin`

Wenn du Colab nutzt, musst du nichts tun. Ansonsten lade dir bitt die Dateien aus den Links oben herunter und bennene sie wie oben beschrieben um.

In [ ]:
if ON_COLAB:
    os.system("test -f emb-glove-de-wiki.txt || wget -O emb-glove-de-wiki.txt https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt")
    os.system("test -f emb-word2vec-de-wiki.txt || wget -O emb-word2vec-de-wiki.txt https://int-emb-word2vec-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt")
    os.system("test -f emb-fasttext-de-wiki.bin || wget -O emb-fasttext-de-wiki.bin https://s3.eu-central-1.amazonaws.com/int-emb-fasttext-de-wiki/20180917/model.bin")

### glove

Das Glove-Modell funktioniert prima, du kannst dir die Daten einfach herunterladen und damit Ähnlichkeiten herausfinden:

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
from gensim.models import KeyedVectors
emb_glove = KeyedVectors.load_word2vec_format("emb-glove-de-wiki.txt", no_header=True)

Ähnlichkeiten kannst du nun ganz einfach ermitteln:

In [ ]:
emb_glove.similar_by_word("apfel")

Die Analogieschlüsse funktionieren nicht so gut wie mit dem Facebook-`fasttext` oben:

In [ ]:
emb_glove.most_similar(positive=["frau", "könig"], negative=["mann"], topn=10)

In [ ]:
emb_glove.most_similar(positive=["frau", "onkel"], negative=["mann"], topn=10)

In [ ]:
# Speicher freigeben für Colab
del emb_glove

## word2vec

Das `word2vec`-Modell ist leider sehr ungünstig kodiert. Zumindest die aktuellen `gensim`-Versionen verstehen die Encodings nicht mehr richtig, daher musst du ein bisschen jonglieren.

In [ ]:
emb_w2v = KeyedVectors.load_word2vec_format("emb-word2vec-de-wiki.txt", no_header=True)

Die Wörter sind "binär" kodiert, das musst du nun simulieren:

In [ ]:
emb_w2v.similar_by_word("b'apfel'")

In [ ]:
emb_w2v.similar_by_word("b'frau'")

Die Analogieschlüsse funktionieren leider nicht besonders gut:

In [ ]:
emb_w2v.most_similar(positive=["b'frau'", "b'k\\xc3\\xb6nig'"], negative=["b'mann'"], topn=10)

In [ ]:
emb_w2v.most_similar(positive=["b'frau'", "b'onkel'"], negative=["b'mann'"], topn=10)

In [ ]:
# Speicher freigeben für Colab
del emb_w2v

### FastText

Das fastText-Modell funktioniert ähnlich gut wie das von Facebook mit Common Crawl trainierte:

In [ ]:
emb_ft = fasttext.load_model("emb-fasttext-de-wiki.bin")

In [ ]:
emb_ft.get_nearest_neighbors("apfel")

Die Analogieschlüsse klappen im Original-Modell deutlich besser:

In [ ]:
emb_ft.get_analogies("frau", "mann", "könig")

In [ ]:
emb_ft.get_analogies("frau", "mann", "onkel")

Allerdings solltest du mit Groß-/Kleinschreibung sehr vorsichtig sein, hier wurde nicht ganz sauber konvertiert:

In [ ]:
emb_ft.get_nearest_neighbors("Apfel")

In [ ]:
# Speicher freigeben für Colab
del emb_ft

## Englischsprachige Embeddings

Google stellt auch vortrainierte Embeddings zur Verfügung, die allerdings schon etwas älter sind: https://github.com/mmihaltz/word2vec-GoogleNews-vectors. Wenn du lokal arbeitest, lade dir die Wortvektoren bitte herunter. Unter Colab musst du selbst nichts machen.

Sie lassen sich ohne Weiteres importieren:

In [ ]:
if ON_COLAB:
    os.system("test -f GoogleNews-vectors-negative300.bin || wget https://figshare.com/ndownloader/files/10798046 && mv 10798046 GoogleNews-vectors-negative300.bin")

In [ ]:
google_w2v = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

Betrachtet nun die Ähnlichkeiten von `woman`:

In [ ]:
google_w2v.similar_by_word("woman")

Die Analogieschlüsse funktionieren relativ gut:

In [ ]:
google_w2v.most_similar(positive=["woman", "king"], negative=["man"], topn=10)

In [ ]:
google_w2v.most_similar(positive=["woman", "uncle"], negative=["man"], topn=10)

## Vorsicht bei externen Embeddings

Externe Embeddings sind sehr nützlich, weil du dich nicht um Trainingsmengen kümmern musst. Du solltest allerdings etwas vorsichtig sein - nicht alle sind mit gleich guter Qualität trainiert. Insbesonders bei der Vorverarbeitung der Daten sind erhebliche Unterschiede vorhanden, das betrifft ganz speziell auch die Tokenisierung und die Groß-/Kleinschreibung.

Wenn du das berücksichtigst und deine Anwendungsfälle daraufhin überprüfst, steht dir eine große, direkt verwendbare Datenmenge zur Verfügung!